In [ ]:
import time
import sys
import os
import numpy as np

%load_ext autoreload
%autoreload 2

import xfmkit.entry_processed as entry_processed
import xfmkit.utils as utils
import xfmkit.argops as argops
import xfmkit.clustering as clustering
import xfmkit.visualisations as vis
import xfmkit.processops as processops
import xfmkit.structures as structures
import xfmkit.config as config

#add the parent folder to sys path so we can import from within the notebook subfolder
#sys.path.insert(0,'..')

print(sys.path)

#args = ' -d /home/lachlan/CODEBASE/xfmkit/data/processed_maps/carlos_full -n 2 -fc'
args_in = ["-d", "/home/lachlan/CODEBASE/xfmkit/data/processed_maps/nf_demo_short" ]

PACKAGE_CONFIG='xfmkit/config.yaml'

In [ ]:
pxs, embedding, categories, classavg, palette = entry_processed.read_processed(args_in)

In [ ]:
config.get('logging','log_file')

In [ ]:
i=11
print(pxs.labels)
print(pxs.labels[i])
print(pxs.weights[i])
np.mean(pxs.weighted.d[:,i]), np.max(pxs.weighted.d[:,i])


In [ ]:
fig = vis.tricolour_pixelset("Cu", "Ca", "S", pxs)

In [ ]:
import xfmkit.tabular as tabular

df = tabular.get_df(classavg, pxs.labels)

IGNORE_LINES=['sum','Back','Compton','Mo','MoL']

N_MAJORS=3

labels = df.columns.values.tolist()

class_majors = []

df_=df.copy()

for i in range(len(df.index)):
    row_ = df_.iloc[i].sort_values(ascending=False)

    #filter method not working as row_ is not a df anymore and thus does not have labels
    #use ignore instead of filter for now, TO-DO fix this
    drop_filter = row_.filter(IGNORE_LINES)

    row_ = row_.drop(IGNORE_LINES, errors='ignore')

    #majors_ = row_.index[0:N_MAJORS].tolist()

    majors_=[]

    for i in range(len(row_.index)):
        if len(majors_) == 0 or (row_[i] > 50000):# and row_[i] > row_[majors_[-1]]/10): 
                #majors_.append(row_.index[i])
                #with pct values:
                majors_.append(str(row_.index[i])+str(int(round(row_[i]/10000,0))))

    class_majors.append(majors_)

class_majors
